In [1]:
import pandas as pd
import os

In [2]:
import chardet 

In [3]:
import pycountry 

In [4]:
from fuzzywuzzy import fuzz, process

c:\Users\samtg\AppData\Local\Programs\Python\Python311\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Define a function to read in and concat the regional election vote totals

In [5]:
def read_and_join(subfolder):
    notebook_dir = os.path.dirname(os.path.abspath('__file__'))
    data_folder_relative = os.path.join('..', '..', 'data', 'raw', subfolder)
    data_folder_absolute = os.path.abspath(data_folder_relative)
    
    excel_files = [file for file in os.listdir(data_folder_absolute) if file.endswith('.xlsx')]

    dataframes = []

    for excel_file in excel_files:
        excel_path = os.path.join(data_folder_absolute, excel_file)
        print(f"Reading {excel_path}")
        
        country_abbreviation = excel_file[:3] 
        
        df = pd.read_excel(excel_path, sheet_name='votes')  
        df['country_abbrev'] = country_abbreviation  
        dataframes.append(df)

    if dataframes:
        combined_df = pd.concat(dataframes, ignore_index=True)
        return combined_df
    else:
        print("No Excel files found or error reading files.")
        return None


Another to read in the party info for joining 

In [6]:
def read_and_join_meta(subfolder):
    notebook_dir = os.path.dirname(os.path.abspath('__file__'))
    data_folder_relative = os.path.join('..', '..', 'data', 'raw', subfolder)
    data_folder_absolute = os.path.abspath(data_folder_relative)
    
    excel_files = [file for file in os.listdir(data_folder_absolute) if file.endswith('.xlsx')]

    dataframes = []

    for excel_file in excel_files:
        excel_path = os.path.join(data_folder_absolute, excel_file)
        print(f"Reading {excel_path}")
        
        country_abbreviation = excel_file[:3]  # Extract the first three characters as abbreviation
        
        df = pd.read_excel(excel_path)
        df.columns = df.columns.str.lower()  # Convert column names to lowercase
        
        # Unify "party" and "party name" columns into a single column "party"
        if "party name" in df.columns:
            df["party"] = df["party name"]
            df = df.drop(columns=["party name"])
        
        df['country_abbrev'] = country_abbreviation  # Add the abbreviation as a new column
        dataframes.append(df)
    
    if dataframes:
        combined_df = pd.concat(dataframes, ignore_index=True)
        return combined_df
    else:
        print("No Excel files found or error reading files.")
        return None


In [7]:
party_info = read_and_join_meta('regional_election')

Reading c:\Users\samtg\github\subnational_inequality\data\raw\regional_election\AUT_reg_1945-2015.xlsx
Reading c:\Users\samtg\github\subnational_inequality\data\raw\regional_election\BEL_reg_1974-2014.xlsx
Reading c:\Users\samtg\github\subnational_inequality\data\raw\regional_election\DEN_reg_1946-2013.xlsx
Reading c:\Users\samtg\github\subnational_inequality\data\raw\regional_election\FRA_reg_1986-2015.xlsx
Reading c:\Users\samtg\github\subnational_inequality\data\raw\regional_election\GER_reg_1946-2015.xlsx
Reading c:\Users\samtg\github\subnational_inequality\data\raw\regional_election\ITA_reg_1947-2015.xlsx
Reading c:\Users\samtg\github\subnational_inequality\data\raw\regional_election\POR_ACO_MAD_reg_1976-2015.xlsx
Reading c:\Users\samtg\github\subnational_inequality\data\raw\regional_election\SPA_reg_1980-2015.xlsx
Reading c:\Users\samtg\github\subnational_inequality\data\raw\regional_election\UKI_reg_1945-2012.xlsx


In [8]:
raw_reg = read_and_join('regional_election')

Reading c:\Users\samtg\github\subnational_inequality\data\raw\regional_election\AUT_reg_1945-2015.xlsx
Reading c:\Users\samtg\github\subnational_inequality\data\raw\regional_election\BEL_reg_1974-2014.xlsx
Reading c:\Users\samtg\github\subnational_inequality\data\raw\regional_election\DEN_reg_1946-2013.xlsx
Reading c:\Users\samtg\github\subnational_inequality\data\raw\regional_election\FRA_reg_1986-2015.xlsx
Reading c:\Users\samtg\github\subnational_inequality\data\raw\regional_election\GER_reg_1946-2015.xlsx
Reading c:\Users\samtg\github\subnational_inequality\data\raw\regional_election\ITA_reg_1947-2015.xlsx
Reading c:\Users\samtg\github\subnational_inequality\data\raw\regional_election\POR_ACO_MAD_reg_1976-2015.xlsx
Reading c:\Users\samtg\github\subnational_inequality\data\raw\regional_election\SPA_reg_1980-2015.xlsx
Reading c:\Users\samtg\github\subnational_inequality\data\raw\regional_election\UKI_reg_1945-2012.xlsx


In [9]:
def add_single_to_dataframe(existing_df, new_excel_path):
    new_df = pd.read_excel(new_excel_path)
    combined_df = pd.concat([existing_df, new_df], ignore_index=True)
    return combined_df

In [56]:
save_folder_relative = os.path.join('..', '..', 'data', 'cleaned')
save_folder_absolute = os.path.abspath(save_folder_relative)

Pivot the vote df longer to join

In [10]:
to_drop = ['elec_day','elec_month','turnout','seats']

In [11]:
raw_reg = raw_reg.drop(columns = to_drop)

In [12]:
id_columns = ['country','country_abbrev','region', 'elec_year', 'electorate','valid','votes']

In [13]:
melted_df = pd.melt(raw_reg, id_vars=id_columns, var_name='party', value_name='vote_count')

In [15]:
melted_df = melted_df.rename(columns = {'party':'party_abbrev'})

#### Pull out France to join with LISSY

In [17]:
melted_france = melted_df[melted_df['country'] == 'France']

In [19]:
party_info_france = party_info[party_info['country_abbrev'] == 'FRA']

In [49]:
party_info_france[party_info_france['abbreviation'] == 'FN']

,abbreviation,notes,party,country_abbrev
484,FN,NaN,Front National,FRA


In [50]:
melted_france[melted_france['party_abbrev'] == 'FN']

,country,country_abbrev,region,elec_year,electorate,valid,votes,party_abbrev,vote_count
226484,France,FRA,Alsace,1986,1037551.0,743945.0,797436.0,FN,96882.0
226485,France,FRA,Aquitaine,1986,1926640.0,1479128.0,1552143.0,FN,103757.0
226486,France,FRA,Auvergne,1986,946332.0,726599.0,759319.0,FN,43106.0
226487,France,FRA,Basse Normandie,1986,954590.0,703432.0,742693.0,FN,51193.0
226488,France,FRA,Bourgogne,1986,1092633.0,799902.0,838899.0,FN,67369.0
...,...,...,...,...,...,...,...,...,...
226660,France,FRA,Languedoc-Roussillon - Midi-Pyrénées,2015,4121310.0,2439233.0,2556086.0,FN,826114.0
226661,France,FRA,Nord-Pas-de-Calais - Picardie,2015,4237939.0,2405002.0,2595417.0,FN,1015662.0
226662,France,FRA,Normandie,2015,2390425.0,1360545.0,1412207.0,FN,374142.0
226663,France,FRA,Pays de la Loire,2015,2664589.0,1452527.0,1525441.0,FN,286711.0


In [47]:
melted_france.tail(15)

,country,country_abbrev,region,elec_year,electorate,valid,votes,party_abbrev,vote_count
2942688,France,FRA,Provence Alpes Cote d'Azur,2010,3347091.0,1694168.0,1747393.0,WP,NaN
2942689,France,FRA,Rhone Alpes,2010,4094267.0,1959180.0,2027120.0,WP,NaN
2942690,France,FRA,Alsace - Champagne-Ardenne - Lorraine,2015,3885868.0,2189981.0,2293374.0,WP,NaN
2942691,France,FRA,Aquitaine - Limousin - Poitou-Charentes,2015,4268772.0,2343298.0,2466376.0,WP,NaN
2942692,France,FRA,Auvergne - Rhône-Alpes,2015,5310464.0,2958455.0,3063198.0,WP,NaN
2942693,France,FRA,Bourgogne - Franche-Comté,2015,1996576.0,1162045.0,1220862.0,WP,NaN
2942694,France,FRA,Bretagne,2015,2421628.0,1304825.0,1377445.0,WP,NaN
2942695,France,FRA,Centre-Val de Loire,2015,1817519.0,1028275.0,1076416.0,WP,NaN
2942696,France,FRA,Corse,2015,229824.0,149526.0,154042.0,WP,NaN
2942697,France,FRA,Île-de-France,2015,7086172.0,3720165.0,3859296.0,WP,NaN


In [37]:
joined_france = melted_france.merge(party_info_france, left_on= 'party_abbrev', right_on= 'abbreviation', how = 'left')

In [57]:
joined_france.to_csv(save_folder_absolute + '/france_party.csv')

Join the two dfs

In [78]:
joined_df = melted_df.merge(party_info, how = 'left', left_on = ['country_abbrev','party_abbrev'], right_on = ['country_abbrev','abbreviation'])

In [79]:
joined_df

,country,country_abbrev,region,elec_year,electorate,valid,votes,party_abbrev,vote_count,abbreviation,notes,party
0,Austria,AUT,Burgenland,1945,137222.0,131597.0,132262.0,AHS,NaN,AHS,NaN,Aktive Heimat - Sozialisten
1,Austria,AUT,Burgenland,1949,169836.0,164398.0,165432.0,AHS,NaN,AHS,NaN,Aktive Heimat - Sozialisten
2,Austria,AUT,Burgenland,1953,176396.0,167514.0,169787.0,AHS,NaN,AHS,NaN,Aktive Heimat - Sozialisten
3,Austria,AUT,Burgenland,1956,175955.0,164950.0,167734.0,AHS,NaN,AHS,NaN,Aktive Heimat - Sozialisten
4,Austria,AUT,Burgenland,1960,178255.0,162664.0,165183.0,AHS,NaN,AHS,NaN,Aktive Heimat - Sozialisten
...,...,...,...,...,...,...,...,...,...,...,...,...
2943403,United Kingdom,UKI,Northern Ireland,1982,1048807.0,633120.0,653450.0,WP,17216.0,WP,NaN,Workers Party
2943404,United Kingdom,UKI,Northern Ireland,1998,1178556.0,807683.0,824391.0,WP,1989.0,WP,NaN,Workers Party
2943405,United Kingdom,UKI,Northern Ireland,2003,1097526.0,692028.0,702249.0,WP,1881.0,WP,NaN,Workers Party
2943406,United Kingdom,UKI,Northern Ireland,2007,1107904.0,690313.0,696538.0,WP,975.0,WP,NaN,Workers Party


Join with Manifesto Data to get party family. 

In [84]:
crosswalk_relative = os.path.join('..', '..', 'data', 'raw', 'crosswalks')
crosswalks_folder_absolute = os.path.abspath(crosswalk_relative)

In [86]:
csv_file_name = 'ches_pfid_manifesto.csv'
csv_file_path = os.path.join(crosswalks_folder_absolute, csv_file_name)

In [93]:
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
        return result['encoding']

crosswalk_relative = os.path.join('..', '..', 'data', 'raw', 'crosswalks')
crosswalks_folder_absolute = os.path.abspath(crosswalk_relative)

csv_file_name = 'ches_pfid_manifesto.csv'
csv_file_path = os.path.join(crosswalks_folder_absolute, csv_file_name)

detected_encoding = detect_encoding(csv_file_path)
print("Detected encoding:", detected_encoding)

try:
    crosswalk_df = pd.read_csv(csv_file_path, encoding=detected_encoding)
    print("CSV file read successfully:", crosswalk_df)
except UnicodeDecodeError:
    print("Unable to read CSV file with detected encoding.")

Detected encoding: ISO-8859-1
CSV file read successfully:     country.x dataset_key.x  dataset_party_id.x name_short.x  \
0         ALB     manifesto               75951       OMONIA   
1         ALB     manifesto               75624           PD   
2         ALB     manifesto               75722         PRSH   
3         ALB     manifesto               75220          PPS   
4         AUT     manifesto               42710         BZï¿   
..        ...           ...                 ...          ...   
390       GBR     manifesto               51421      LibDems   
391       GBR     manifesto               51901           PC   
392       GBR     manifesto               51902          SNP   
393       GBR     manifesto               51210           SF   
394       GBR     manifesto               51951         UKIP   

                                         name.x  \
0    Partia Bashkimi pï¿½ï¿½r tï¿½ï¿½ Drejtat e   
1             Partia Demokratike e Shqipï¿½ï¿½r   
2                  P

In [115]:
joined_df.columns

Index(['country', 'country_abbrev', 'region', 'elec_year', 'electorate',
       'valid', 'votes', 'party_abbrev', 'vote_count', 'abbreviation', 'notes',
       'party'],
      dtype='object')

In [113]:
crosswalk_df['country_name'] = crosswalk_df['country.x'].apply(lambda x: pycountry.countries.get(alpha_3=x).name if pycountry.countries.get(alpha_3=x) else x)

In [116]:
joined_df['country_name'] = joined_df['country_abbrev'].apply(lambda x: pycountry.countries.get(alpha_3=x).name if pycountry.countries.get(alpha_3=x) else x)

In [119]:
joined_df['country_name'] 

0                 Austria
1                 Austria
2                 Austria
3                 Austria
4                 Austria
                ...      
2943403    United Kingdom
2943404    United Kingdom
2943405    United Kingdom
2943406    United Kingdom
2943407    United Kingdom
Name: country_name, Length: 2943408, dtype: object

In [118]:
joined_df['country_name'] = joined_df['country_name'].replace('UKI', 'United Kingdom')

In [114]:
crosswalk_df['country_name']

0             Albania
1             Albania
2             Albania
3             Albania
4             Austria
            ...      
390    United Kingdom
391    United Kingdom
392    United Kingdom
393    United Kingdom
394    United Kingdom
Name: country_name, Length: 395, dtype: object

In [123]:
def fuzzy_match(query, choices):
    return process.extractOne(query, choices, scorer=fuzz.partial_ratio)[0]

In [129]:
# Specify the list of columns you want to keep
columns_to_keep = ['dataset_key.x', 'dataset_party_id.x', 'name_english.x', 'dataset_key.y', 'dataset_party_id.y', 'name_english.y', 'country_name']

# Select only the specified columns
crosswalk_df = crosswalk_df[columns_to_keep]

In [134]:
joined_df['country_name'] = joined_df['country_name'].astype(str)
crosswalk_df['country_name'] = crosswalk_df['country_name'].astype(str)

C:\Users\samtg\AppData\Local\Temp\ipykernel_2484\2060973019.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crosswalk_df['country_name'] = crosswalk_df['country_name'].astype(str)


In [148]:
joined_df.head(5)

,country,country_abbrev,region,elec_year,electorate,valid,votes,party_abbrev,vote_count,abbreviation,notes,party,country_name
0,Austria,AUT,Burgenland,1945,137222.0,131597.0,132262.0,AHS,NaN,AHS,NaN,Aktive Heimat - Sozialisten,Austria
1,Austria,AUT,Burgenland,1949,169836.0,164398.0,165432.0,AHS,NaN,AHS,NaN,Aktive Heimat - Sozialisten,Austria
2,Austria,AUT,Burgenland,1953,176396.0,167514.0,169787.0,AHS,NaN,AHS,NaN,Aktive Heimat - Sozialisten,Austria
3,Austria,AUT,Burgenland,1956,175955.0,164950.0,167734.0,AHS,NaN,AHS,NaN,Aktive Heimat - Sozialisten,Austria
4,Austria,AUT,Burgenland,1960,178255.0,162664.0,165183.0,AHS,NaN,AHS,NaN,Aktive Heimat - Sozialisten,Austria


In [149]:
joined_df['country'].value_counts()

country
Denmark           694200
Italy             538272
Germany           459240
France            386616
Spain             371664
Austria           299040
United Kingdom     79032
Belgium            57672
Portugal           44856
Germany            12816
Name: count, dtype: int64

In [150]:
to_keep = ['Italy','Spain','France','Germany','Denmark','Austria','United Kingdom','Belgium','Portugal']

In [153]:
crosswalk_df['country_name'].value_counts()

country_name
Italy             36
Spain             17
Belgium           12
France            12
Denmark           11
United Kingdom     9
Portugal           8
Austria            7
Germany            6
Name: count, dtype: int64

In [151]:
crosswalk_df = crosswalk_df[crosswalk_df['country_name'].isin(to_keep)]

In [152]:
merged_df = joined_df.merge(crosswalk_df, how = 'inner', left_on = 'country',right_on = 'country_name')

MemoryError: Unable to allocate 1020. MiB for an array with shape (3, 44582592) and data type float64